In [56]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import os
#os.chdir("/home/kaborg15/Vibrent_Dataset_Collection/")
os.chdir(r"C:\Users\kaborg15\Python_projects\Vibrent_Dataset_Collection")

from src.collect_database_data import construct_user_orders, format_outfit_array, get_outfit_pictures
from src import load_dataframes
from resources.constants import *
from src import build_dataset

#orders_df, spot_orders_df, outfits_df, pictures_df = build_dataset.get_dataframes(load_local_dataframes=True, include_tag_data=True)
orders_df, spot_orders_df, outfits_df, pictures_df  = build_dataset.build_dataset(load_local_dataframes=True)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Finished loading dataframes.
Finished constructing user activity triplets.
Finished constructing spot rentals.
Finished constructing outfit data.
Finished constructing picture triplets.
Finished constructing third chance outfit data.


In [57]:
import pandas as pd

#third_chance_df = pd.read_excel(THIRD_CHANCE_RAW_PATH)
third_chance_df = pd.read_csv(THIRD_CHANCE_CSV_PATH, sep=CSV_SEPARATOR)
third_chance_df.head(2)

,date_added,id,name,brand,owner,condition,condition_desctription,retail_price,tc_price,sold,varetelling
0,2023-11-23,-,Blue Scarf 1,Holzweiler,Holzweiler,6 - Redesigned,Ødelagt på armene. Stygg slitasje,2500,650.0,True,False
1,2023-11-23,-,Blue Scarf 2,Holzweiler,Holzweiler,6 - Redesigned,NaN,2800,750.0,True,False


In [58]:
def column_name_assignment(column_name):
    return column_name.lower().replace(" ", "_")

third_chance_df = third_chance_df.drop(columns=THIRD_CHANCE_DROP_COLUMNS, errors="ignore")
third_chance_df = third_chance_df.rename(mapper=column_name_assignment, axis=1)
third_chance_df = third_chance_df.dropna(subset=["id", "name", "owner", "brand"], how="all")
third_chance_df = third_chance_df.dropna(subset=["condition", "retail_price", "tc_price"], how="all")
third_chance_df

,date_added,id,name,brand,owner,condition,condition_desctription,retail_price,tc_price,sold,varetelling
0,2023-11-23,-,Blue Scarf 1,Holzweiler,Holzweiler,6 - Redesigned,Ødelagt på armene. Stygg slitasje,2500,650.0,True,False
1,2023-11-23,-,Blue Scarf 2,Holzweiler,Holzweiler,6 - Redesigned,NaN,2800,750.0,True,False
2,2023-11-23,-,Green Scarf 1,Holzweiler,Holzweiler,5 - Damaged,Slitt og går ikke an å vaskes,12000,2400.0,False,False
3,2023-11-23,-,Green Scarf 2,Holzweiler,Holzweiler,5 - Damaged,Masse flekker,10000,1800.0,False,False
4,2023-11-23,-,Green Scarf 3,Holzweiler,Holzweiler,1 - As new,tror det bare er fordi den er en singlet,500,250.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...
1291,2022-09-15,outfit.aad989fc0422b4eb,XS2 Winslow Dress Azalea,Reformation,FJONG,2 - Very good condition,NaN,700,290.0,False,False
1292,2022-07-07,utfit.b2aa8b59d1644a0090ab62a119e1ea41,Shiny Grape Leaf Trousers,Mette Møller,Mette Møller,2 - Very good condition,NaN,1200,390.0,False,False
1293,2023-09-19,NaN,Long Black Dress,Second Female,FJONG,2 - Very good condition,NaN,800,300.0,False,False
1294,2023-09-19,NaN,Green Jumpsuit,Hofmann Copenhagen,FJONG,2 - Very good condition,NaN,900,320.0,False,False


In [59]:
def convert_date_added(date_string):
    if date_string == "-":
        return None
    else:
        return pd.to_datetime(date_string)

third_chance_df["date_added"] = third_chance_df["date_added"].apply(convert_date_added)

In [61]:
third_chance_df["date_added"].isna().sum()

263

In [47]:
#"L Helmine Skirt" in outfits_df["name"].values
from Levenshtein import distance

def find_n_most_similar_names(name, names, n=5):
    return sorted(names, key=lambda x: distance(name, x))[:n]

undefined_id_outfits = third_chance_df[third_chance_df["id"] == "-"]

for undefined_name in undefined_id_outfits["name"].values:
    #print(undefined_name, undefined_name in outfits_df["name"].values)
    print(undefined_name, find_n_most_similar_names(undefined_name, outfits_df["name"].values, n=5))

Blue Scarf 1 ['Blue Scarf Skirt', 'Bali Shirt ', 'Blue Neckwarmer', 'Lua Skirt ', 'Blue Gown']
Blue Scarf 2 ['Blue Scarf Skirt', 'Bali Shirt ', 'Blue Neckwarmer', 'Lua Skirt ', 'Blue Gown']
Green Scarf 1 ['Green Sweater ', 'Orelia Scarf', 'Green Sweater ', 'Green Sweater ', 'Green Blazer']
Green Scarf 2 ['Green Sweater ', 'Orelia Scarf', 'Green Sweater ', 'Green Sweater ', 'Green Blazer']
Green Scarf 3 ['Green Sweater ', 'Orelia Scarf', 'Green Sweater ', 'Green Sweater ', 'Green Blazer']
Green Scarf 4 ['Green Sweater ', 'Orelia Scarf', 'Green Sweater ', 'Green Sweater ', 'Green Blazer']
Green Scarf 5 ['Green Sweater ', 'Orelia Scarf', 'Green Sweater ', 'Green Sweater ', 'Green Blazer']
M Emma Pants glatt 1 ['Pinja Pants Black', 'Dazy F Pants Black ', 'Macie Pants Black', 'Dazy Pants Black ', 'Zinnia Pants Pecan ']
M Emma Pants glatt 2 ['Pinja Pants Black', 'Dazy F Pants Black ', 'Macie Pants Black', 'Dazy Pants Black ', 'Zinnia Pants Pecan ']
M Emma LS Sweater ['Marianne Sweater', 'Plo

In [3]:
# import pandas as pd
# from src import retrieve_image_bucket_data
# from resources.constants import *

# picture_triplets_df = pd.read_csv(DATASET_FOLDER + "picture_triplets.csv", sep=";")

# dataset_picture_ids = picture_triplets_df["picture.id"].unique()
# #print(len(dataset_picture_ids), len(pictures_df["id"].unique()))
# IMAGES_FOLDER = "resources/data/dataset/images/"
# existing_images = os.listdir(IMAGES_FOLDER)
# dataset_picture_ids = list(set(dataset_picture_ids) - set(existing_images))
# retrieve_image_bucket_data.download_to_folder(dataset_picture_ids, IMAGES_FOLDER)
# from src import create_image_embeddings
# import pandas as pd
# from resources.constants import *

# picture_triplets_df = pd.read_csv(DATASET_FOLDER + "picture_triplets.csv", sep=";")
# picture_triplets_df[:10]
# embeddings_df = create_image_embeddings.get_df_image_embeddings(picture_triplets_df[:10], DATASET_IMAGE_EMBEDDINGS_FOLDER, config_class=create_image_embeddings.EfficientNet_V2_L_final)
# embeddings_df["embeddings"].values[0].shape

KeyboardInterrupt: 